In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

/usr/local/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
NUM_FEATURES = 11
NUM_OUTPUTS = 1

In [3]:
dat = pd.read_csv('datasets/winequality-red.csv')
X_total, Y_total = dat.values[:, 0:-1], dat.values[:, [-1]]
X_train, X_test, Y_train, Y_test = train_test_split(X_total, Y_total, test_size=0.2)

# TensorBoard

![](https://www.tensorflow.org/images/mnist_tensorboard.png)

- [TensorBoard: Visualizing Learning  |  TensorFlow](https://www.tensorflow.org/get_started/summaries_and_tensorboard)
- [텐서보드 · 텐서플로우 문서 한글 번역본](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/summaries_and_tensorboard/)

```bash
tensorboard --logdir=path/to/log-directory
```

In [4]:
with tf.name_scope('Inputs'):
    X = tf.placeholder(tf.float32, shape=[None, NUM_FEATURES])

with tf.name_scope('Targets'):
    Y = tf.placeholder(tf.float32, shape=[None, NUM_OUTPUTS])

with tf.name_scope('Weights'):
    W = tf.Variable(tf.random_normal([NUM_FEATURES, NUM_OUTPUTS]), name='Weights')

with tf.name_scope('Bias'):
    b = tf.Variable(tf.random_normal([NUM_OUTPUTS]), name='Bias')

In [5]:
with tf.name_scope('Hypothesis'):
    hypothesis = tf.matmul(X, W) + b

with tf.name_scope('Train'):
    with tf.name_scope('Cost'):
        cost = tf.reduce_mean(tf.square(hypothesis - Y))
        tf.summary.scalar('Cost', cost)

    with tf.name_scope('Optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-4)
        
    with tf.name_scope('Gradient'):
        gradient = optimizer.compute_gradients(cost, [W])
        tf.summary.scalar('Gradient', gradient)

    train = optimizer.minimize(cost)

In [6]:
import time
from pathlib import Path

LOGPATH = Path('./logs')
timestamp_log = time.strftime('%y%m%d%H%M')

logdir = str(LOGPATH / f'lin-reg-{timestamp_log}')

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(logdir, sess.graph)
merged = tf.summary.merge_all()

In [8]:
costs = []
for step in range(2001):
    _, summary = sess.run([train, merged], feed_dict={X: X_train, Y: Y_train})
    writer.add_summary(summary, step)
    writer.flush()

InvalidArgumentError: tags and values not the same shape: [] != [1,2,11,1] (tag 'Train/Gradient/Gradient')
	 [[Node: Train/Gradient/Gradient = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Train/Gradient/Gradient/tags, Train/Gradient/Gradient/values)]]

Caused by op 'Train/Gradient/Gradient', defined at:
  File "/usr/local/anaconda3/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda3/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda3/envs/tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File 

In [ ]:
writer.close()
sess.close()